# Gradient Descent

# Limitations of the Normal Equation

We have seen that we can analytically determine the optimum for a linear regression function using the Normal Equation:

$$ \theta = (X^TX)^{-1} \cdot X^Ty $$

Although the Normal Equation is a very fast way to find optimal linear regression functions, it has some limitations in practice:

- linear functions
- least squares cost function
- $X^T \cdot X$ must be invertible (no colinearity)
- Numerical unstable when close to colinear
- Computing the inverse has a computational upperbound of $O(n^3)$ where n is the number of features/columns. Therefore the Normal Equation is not efficient for large values of $n$.
- When the data set does not fit in RAM, computation becomes difficult and inefficient.

There are several alternatives to approximate an optimal function for a given true function. We will focus on Gradient Descent for this purpose, since it is the most general en widely used optimization algorithm.

# Gradient Descent

According to this method, the coefficients $\theta$ are adjusted iteratively towards the minimum. To determine the direction for adjusting $\theta$ we use the partial derivatives of the cost function. An advantage of Gradient Descent is that the approach is applicable to any cost function.

# Update Rule

The basis for Gradient Descent is an update rule that adjusts the coefficients towards the minimum. We use a general update rule, in which $\theta_j$ is a coefficient that is updated, $J(\theta)$ is the cost function (parameterised by $\theta$) and $\alpha$ is the 'learning rate'.

$$ \theta_j = \theta_j - \alpha \frac{\delta}{\delta \theta_j}J(\theta) $$

In the update rule, we can explain the minus-sign by looking at the derivative. Imagine the cost function as being bowl-shaped and our objective is to find the minimum. If the gradient is negative the minimum is to the right, hence we should increase $\theta$. If the gradient is positive, the minimum is to the left, hence we shoudl decrease $\theta$.

The learning rate $\alpha$ controls the size of the update steps. We will later discuss the problem of setting the learning rate to a proper value, but for now we will just give a suitable value.

We have discussed the cost function $J(\theta)$ that we can use to analytically determine the optimal $\theta$. Because we use the derivative of the cost function to find the optimum, it is actually more common to add a factor $\frac{1}{2m}$. For the optimization problem there is no difference, but this addition shortens the gradient and thus the update rule. 

$$ J(\theta) = \frac{1}{2m} \sum_{i = 1}^m (\widehat{y^{(i)}} - y^{(i)})^2 = \frac{1}{2m} \sum_{i = 1}^m (\theta_1 \cdot x^{(i)} + \theta_0 - y^{(i)})^2 $$  

We now derive the partial derivative by applying the chain-rule:

$$ \frac{\delta J(\theta)}{\delta \theta_0} = \frac{1}{m} \sum_{i=1}^m (\theta_1 \cdot x^{(i)} + \theta_0 - y^{(i)}) \cdot 1 $$

$$ \frac{\delta J(\theta)}{\delta \theta_1} = \frac{1}{m} \sum_{i=1}^m (\theta_1 \cdot x^{(i)} + \theta_0 - y^{(i)}) \cdot x^{(i)} $$

And the update rules then become:

$$ \theta_0 := \theta_0 - \alpha \cdot \frac{1}{m} \sum_{i=1}^m (\theta_1 \cdot x^{(i)} + \theta_0 - y^{(i)}) $$
$$ \theta_1 := \theta_1 - \alpha \cdot \frac{1}{m} \sum_{i=1}^m (\theta_1 \cdot x^{(i)} + \theta_0 - y^{(i)}) \cdot x^{(i)} $$
 

# Vectorization

Although we can compute the updates of all training samples in a loop, this would be very slow in Python. Alternatively, we can rewrite these update rules as dot matrix multiplications, to make the code run much faster. The process of rewriting the agorithm into matrix operations is often callen vectorization.

To vectorize our code, we will first rewrite our cost function and update rules using a dot product between $theta$ and $x$. Recall that we have initialize written our hypothesis as $\theta^T \cdot x$:

$$ \theta_0 := \theta_0 - \alpha \cdot \frac{1}{m} \sum_{i=1}^m (\theta^T \cdot x^{(i)} - y^{(i)}) \cdot x^{(i)}_0 $$

$$ \theta_1 := \theta_1 - \alpha \cdot \frac{1}{m} \sum_{i=1}^m (\theta^T \cdot x^{(i)} - y^{(i)}) \cdot x^{(i)}_1 $$

We can then replace the summation by yet another dot product and take all update rules together in one go:

$$ \theta := \theta - \frac{\alpha}{m} X^T \cdot (X \cdot \theta - y) $$

# Data

For starters, we read the Wine data set like before, which gives us an input matrix X and a target vector y.

In [1]:
from ml import *
data = advertising_sales_tv(bias=True, column_y = True)
X = data.train_X
y = data.train_y

In [2]:
X

array([[  1. ,  93.9],
       [  1. ,  75.1],
       [  1. ,   4.1],
       [  1. , 195.4],
       [  1. , 261.3],
       [  1. , 276.9],
       [  1. , 141.3],
       [  1. ,   0.7],
       [  1. , 228.3],
       [  1. , 171.3],
       [  1. , 112.9],
       [  1. , 187.9],
       [  1. , 109.8],
       [  1. ,   8.4],
       [  1. , 255.4],
       [  1. ,   7.8],
       [  1. , 281.4],
       [  1. , 292.9],
       [  1. , 276.7],
       [  1. , 188.4],
       [  1. , 120.5],
       [  1. , 129.4],
       [  1. , 109.8],
       [  1. ,   5.4],
       [  1. , 293.6],
       [  1. , 219.8],
       [  1. ,  17.2],
       [  1. ,  97.5],
       [  1. , 240.1],
       [  1. , 213.4],
       [  1. ,   8.7],
       [  1. ,  78.2],
       [  1. , 280.2],
       [  1. , 218.5],
       [  1. ,  18.8],
       [  1. , 215.4],
       [  1. , 164.5],
       [  1. ,  62.3],
       [  1. ,  96.2],
       [  1. , 217.7],
       [  1. ,   8.6],
       [  1. , 182.6],
       [  1. , 240.1],
       [  1

# Model

The next thing we need is a vector that contains the parameters for our linear function. In Machine Learning, by default we use $\theta$ (pronounce theta) to hold our parameters.

In [3]:
# create a column vector with 2 zeros: 2 rows, 1 column
𝜃 = np.zeros((2, 1))
𝜃

array([[0.],
       [0.]])

Then $\hat{y} = X \cdot \theta$ will give the estimates in a column vector, which we can compare to $y$. In this case, we will get only zeros, because $\theta_0 = \theta_1 = 0$, and therefore the dot product with whatever values will be zero. We still need to train the model to find the best values for $\theta$.

In [4]:
y_hat = X @ 𝜃
y_hat

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

# Training

We now apply the vectorized update rule we derived above repeatedly and the values of $\theta$ will converge to the optimal value.

We introduce a 'learning rate' to update $\theta$ with some fraction of the gradient to iteratively take a small step in the right direction.

In [4]:
# learning rate
𝛼 = 0.00003

In [5]:
for i in range(500000):
    𝜃 = 𝜃 - 𝛼 * X.T @ (X @ 𝜃 - y) / len(X)
    if i % 10000 == 0:
        print(𝜃)
print(𝜃)

[[0.00090994]
 [0.16114593]]
[[0.99733529]
 [0.08482212]]
[[1.85090769]
 [0.08057229]]
[[2.58184666]
 [0.07693303]]
[[3.20777106]
 [0.07381663]]
[[3.74376841]
 [0.07114796]]
[[4.20275859]
 [0.06886271]]
[[4.5958053 ]
 [0.06690578]]
[[4.93238267]
 [0.06523   ]]
[[5.22060371]
 [0.06379498]]
[[5.4674158 ]
 [0.06256613]]
[[5.67876821]
 [0.06151383]]
[[5.85975545]
 [0.06061272]]
[[6.01474011]
 [0.05984107]]
[[6.147458  ]
 [0.05918028]]
[[6.26110821]
 [0.05861443]]
[[6.35843021]
 [0.05812988]]
[[6.44176987]
 [0.05771494]]
[[6.51313606]
 [0.05735962]]
[[6.57424902]
 [0.05705534]]
[[6.62658182]
 [0.05679478]]
[[6.67139594]
 [0.05657166]]
[[6.70977157]
 [0.05638059]]
[[6.74263374]
 [0.05621698]]
[[6.77077459]
 [0.05607687]]
[[6.79487241]
 [0.05595689]]
[[6.81550808]
 [0.05585414]]
[[6.83317901]
 [0.05576616]]
[[6.84831114]
 [0.05569082]]
[[6.86126923]
 [0.0556263 ]]
[[6.87236561]
 [0.05557106]]
[[6.88186778]
 [0.05552375]]
[[6.89000475]
 [0.05548323]]
[[6.89697269]
 [0.05544854]]
[[6.90293953]


When running the code below, you may notice a few things:

- We are getting an estimation of the optimal coefficients instead of the exact outcome
- It takes a few seconds to converge, which indicates that it may be less efficient for large data sets. Although there are several tricks we can do to speed up learning.